In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
xtrain=pd.read_csv('/kaggle/input/loan-prediction-problem-dataset/train_u6lujuX_CVtuZ9i.csv')

In [ ]:
df=xtrain #Copying original dataset to another df

In [ ]:
df

In [ ]:
df=df.replace(['Male','Female'],[1,0])

In [ ]:
df=df.replace(['Yes','No'],[1,0])

In [ ]:
df['Loan_Status']=df['Loan_Status'].replace(['Y','N'],[1,0])

In [ ]:
df


In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.nunique()

In [ ]:
df['Gender'].value_counts() #11 missing values---bias towards 489

In [ ]:
df['Married'].value_counts() #3 missing values--- replace by 1/ drop them. but i am replacing them

In [ ]:
df['Dependents'].value_counts() #15 missing values---bias towards 345

In [ ]:
df['Dependents']=df['Dependents'].replace(['3+'],[3])

In [ ]:
df

In [ ]:
df['Self_Employed'].value_counts() #32 missing values--- bias towards 500 

In [ ]:
df['Loan_Amount_Term'].value_counts() #14 missing values--- bias towards 512

In [ ]:
df['Credit_History'].value_counts() #50 missing values--- bias towards 475

In [ ]:
15/614,32/614,14/614,50/614

In [ ]:
# Less than 1% missing values are there
# Replacing all by mode is apt solution except for Loan amt which is replaced by mean.
# Apply feature scaling

## Apply imputer. but as of now i am dropping missing values.

## 1.IMPUTER

In [ ]:
(df['Dependents']) # TYpe is object. SO convert into float

In [ ]:
df.Dependents=df.Dependents.apply(pd.to_numeric)

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
imputer=SimpleImputer(missing_values=np.nan, strategy='most_frequent')

In [ ]:
imputer.fit(df.iloc[:,[1,2,3,5,9,10]])
df.iloc[:,[1,2,3,5,9,10]] = imputer.transform(df.iloc[:,[1,2,3,5,9,10]])

In [ ]:
gender_mode=df['Gender'].mode()
employed_mode=df['Self_Employed'].mode()
LoanAmount_mean=df['LoanAmount'].mean()
LoanAmountTerm_mean=df['Loan_Amount_Term'].mean()
credit_mode=df['Credit_History'].mode()
dep_mode=df['Dependents'].mode()



In [ ]:
df.info()

In [ ]:
#Dropping LoanId as it dosen't affect prediction
df=df.drop(columns='Loan_ID')

In [ ]:
df

## Categorical columns are there: Education,Property_Area

In [ ]:
columntitle=["Education","Property_Area","Gender","Married","Dependents","Self_Employed","ApplicantIncome","CoapplicantIncome","LoanAmount","Loan_Amount_Term","Credit_History","Loan_Status"]

In [ ]:
df=df.reindex(columns=columntitle)

In [ ]:
df

In [ ]:
pd.isnull(df).sum() > 0 
#Check nan valuesin any column before label encoding

In [ ]:
imputer_loan=SimpleImputer(missing_values=np.nan,strategy="mean")
imputer_loan.fit(df.iloc[:,[8]])
df.iloc[:,[8]] = imputer_loan.transform(df.iloc[:,[8]])

In [ ]:
from sklearn.preprocessing import OneHotEncoder,LabelEncoder

In [ ]:
label_enc=LabelEncoder()
df['Education']=label_enc.fit_transform(df['Education'])
df['Property_Area']=label_enc.fit_transform(df['Property_Area'])

# Instead of this can apply colmntransformer only


In [ ]:
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer(
    [('onehot', OneHotEncoder(categories='auto'), [0,1])],   # The column numbers to be transformed (here is [0] but can be [0, 1, 3])
    remainder='passthrough'                                         # Leave the rest of the columns untouched
)

In [ ]:
X = ct.fit_transform(df)

In [ ]:
ct


In [ ]:
# 2 diffnames for education and 3 diff categories for propertyname

column_name_afterEncoding=columntitle
column_name_afterEncoding.insert(0,"Education")
column_name_afterEncoding.insert(2,"Property_Area")
column_name_afterEncoding.insert(3,"Property_Area")
print(column_name_afterEncoding)

In [ ]:
encoded_df=pd.DataFrame(X,columns=column_name_afterEncoding)

In [ ]:
encoded_df

In [ ]:
df['Education'].notnull().nunique()

In [ ]:
encoded_df_X=encoded_df.iloc[:,:14]

In [ ]:
encoded_df_X

In [ ]:
encoded_df_Y=encoded_df.iloc[:,-1:]

In [ ]:
encoded_df_Y

# 1.Logistic Regresssion

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:

classifier=LogisticRegression(random_state=0)


In [ ]:
classifier.fit(encoded_df_X,encoded_df_Y)

## Xtest

In [ ]:
xtest=pd.read_csv('/kaggle/input/loan-prediction-problem-dataset/test_Y3wMUE5_7gLdaTN.csv')

In [ ]:
xtest

In [ ]:
xtest.info()

In [ ]:
xtest=xtest.drop(columns=['Loan_ID'])

In [ ]:
xtest['Dependents']=xtest['Dependents'].replace(['3+'],[3])
# xtest['Loan_Status']=xtest['Loan_Status'].replace(['Y','N'],[1,0])
xtest=xtest.replace(['Yes','No'],[1,0])
xtest=xtest.replace(['Male','Female'],[1,0])

In [ ]:
xtest.Dependents=xtest.Dependents.apply(pd.to_numeric)
(xtest['Dependents']) # TYpe is object. SO convert into float

In [ ]:
xtest

In [ ]:
xtest.info()

In [ ]:
# Gender
xtest['Gender'].fillna(gender_mode[0],inplace=True) #[0] is used as series is o/p
# Self_Employed
xtest['Self_Employed'].fillna(employed_mode[0],inplace=True)
# Loan_Amount
xtest['LoanAmount'].fillna(LoanAmount_mean,inplace=True)
# Loan_Amount_Term
xtest['Loan_Amount_Term'].fillna(LoanAmountTerm_mean,inplace=True)
# Credit_History
xtest['Credit_History'].fillna(credit_mode[0],inplace=True)
# Dependents
xtest['Dependents'].fillna(dep_mode[0],inplace=True)


In [ ]:
xtest.info()

In [ ]:
# Rearranging columns acc. to  df
columntitle=["Education","Property_Area","Gender","Married","Dependents","Self_Employed","ApplicantIncome","CoapplicantIncome","LoanAmount","Loan_Amount_Term","Credit_History"]
xtest=xtest.reindex(columns=columntitle)



In [ ]:
xtest

# actually TrANSFORM ONLY TO BE USED NOT FIT_TANSFORM


In [ ]:
# Categorical_Var
xtest['Education']=label_enc.fit_transform(xtest['Education']) #actually TANSFORM ONLY TO BEUSED NOT FIT_TANSFORM
xtest['Property_Area']=label_enc.fit_transform(xtest['Property_Area'])

In [ ]:
xtest

In [ ]:

ct1 = ColumnTransformer(
    [('onehot', OneHotEncoder(categories='auto'), [0,1])],   # The column numbers to be transformed (here is [0] but can be [0, 1, 3])
    remainder='passthrough'                                         # Leave the rest of the columns untouched
)

In [ ]:
X1 = ct1.fit_transform(xtest)


In [ ]:
X1.shape

In [ ]:

Xtest_columntitle=["Education","Property_Area","Gender","Married","Dependents","Self_Employed","ApplicantIncome","CoapplicantIncome","LoanAmount","Loan_Amount_Term","Credit_History"]


In [ ]:
Xtest_column_name_afterEncoding=Xtest_columntitle
Xtest_column_name_afterEncoding.insert(0,"Education")
Xtest_column_name_afterEncoding.insert(2,"Property_Area")
Xtest_column_name_afterEncoding.insert(3,"Property_Area")

In [ ]:
encode_xtest=pd.DataFrame(X1,columns=Xtest_column_name_afterEncoding)

In [ ]:
encode_xtest

In [ ]:
encode_xtest.info()

In [ ]:
x_test_pred=classifier.predict(encode_xtest)

In [ ]:
x_test_pred

In [ ]:
pd.isnull(encode_xtest).sum() > 0 

# No evaluation done. DO it too

# Submission

In [ ]:
p2= pd.DataFrame({'Sur': x_test_pred.astype(int)})

In [ ]:
p2

In [ ]:
p2['Sur'].value_counts()

In [ ]:
p2.to_csv('loan.csv',index=False)


## 2.DROPPING VALUES

In [ ]:
df=df.dropna()

In [ ]:
df=df.reset_index()

In [ ]:
df=df.drop(columns='index')

In [ ]:
df=df.drop(columns='Loan_ID')

## As of now i am also dropping Education,Property area as i am not applying hot encoding. but apply it

In [ ]:
df=df.drop(columns=['Education','Property_Area'])

In [ ]:
df

In [ ]:
x=df.iloc[:,0:9]

In [ ]:
x

In [ ]:
y=df.iloc[:,-1]

In [ ]:
y=pd.DataFrame(y)

In [ ]:
y

## Xtest

In [ ]:
xtest=pd.read_csv('/kaggle/input/loan-prediction-problem-dataset/test_Y3wMUE5_7gLdaTN.csv')

In [ ]:
xtest

In [ ]:
xtest.info()

## Preprocessing xtest

In [ ]:
xtest=xtest.drop(columns=['Loan_ID','Education','Property_Area'])

In [ ]:
xtest

In [ ]:
xtest=xtest.replace(['Male','Female'],[1,0])

In [ ]:
xtest=xtest.replace(['Yes','No'],[1,0])

In [ ]:
xtest=xtest.dropna()

In [ ]:
xtest=xtest.reset_index()

In [ ]:
xtest

In [ ]:
xtest=xtest.drop(columns='index')
# df=df.drop(columns='index')

In [ ]:
xtest['Dependents']=xtest['Dependents'].replace(['3+'],[3])

In [ ]:
x_test_pred=classifier.predict(xtest)

In [ ]:
x_test_pred

In [ ]:
# from sklearn.metrics import confusion_matrix 


In [ ]:
# cm=confusion_matrix()